## Introduction to tensors

In [1]:
import numpy as np

In [15]:
class Tensor(object):
    
    def __init__(self,data,creators=None,creation_op=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_op= creation_op
        self.grad = None
    def backward(self,grad):
        self.grad = grad
        if(self.creation_op =="add"):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
                
    def __add__(self,other):
        return Tensor(self.data+other.data,
                         creators=[self,other],
                        creation_op="add")
    
    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

creating a tensor

In [16]:
x = Tensor([1,2,3,4,5])
print(x)

[1 2 3 4 5]


Perfomring addition operation with two tensors

In [17]:
y = x+x
print(y)

[ 2  4  6  8 10]


Right now there are no gradinets since we did not do a back propogation

In [18]:
print(x.grad)
print(y.grad)


None
None


Here we are adding two vectors to create z this should create a computation graph and have it pointed to z with nodes x and y which edges labeling the creation_op

In [28]:
z = x+y
print(z)
z.backward(Tensor([1,1,1,1,1]))

[ 3  6  9 12 15]


In [29]:
print(x.grad)
print(y.grad)

[1 1 1 1 1]
[1 1 1 1 1]


In [30]:
print(z.creators)

[array([1, 2, 3, 4, 5]), array([ 2,  4,  6,  8, 10])]


In [31]:
print(z.creation_op)

add


The most elegent part of this autograd is that it works recursively as well becuase each vector calls .backward() on all of its self.creators:

In [39]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a+b
f = c+d
g = e+f
g.backward(Tensor(np.array([1,1,1,1,1])))
list=[a,b,c,d,e,f,g]
for i in list:
    print(i,"has gradient",i.grad)

[1 2 3 4 5] has gradient [1 1 1 1 1]
[2 2 2 2 2] has gradient [1 1 1 1 1]
[5 4 3 2 1] has gradient [1 1 1 1 1]
[-1 -2 -3 -4 -5] has gradient [1 1 1 1 1]
[3 4 5 6 7] has gradient [1 1 1 1 1]
[ 4  2  0 -2 -4] has gradient [1 1 1 1 1]
[7 6 5 4 3] has gradient [1 1 1 1 1]


# Updating autograd to support multiuse tensor

In [1]:
import numpy as np


class Tensor(object):

    def __init__(self,data,
                autograd=False,
                creators=None,
                creation_op=None,
                id=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        self.autograd = autograd
        self.childern = {}

        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id

        if(creators is not None):
            for c in creators:
                if(self.id not in c.childern):
                    c.childern[self.id] = 1
                else:
                    c.childern[self.id] += 1
    def all_childern_grads_accounted_for(self):
        for id,cnt in self.childern.items():
            if(cnt !=0 ):
                return False
        return True

    def backward(self,grad=None,grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                if(self.childern[grad_origin.id]==0):
                    raise Exception("cannto backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

        if(self.grad is None):
            self.grad = grad
        else:
            self.grad += grad
        if(self.creators is not None and
            (self.all_childern_grads_accounted_for()or grad_origin is None)):
                if(self.creation_op =="add"):
                    self.creators[0].backward(grad)
                    self.creators[1].backward(grad)

    def __add__(self,other):
        if(self.autograd and other.autograd):
            return Tensor(self.data+other.data,
                          autograd=True,
                      creators=[self,other],
                      creation_op="add")
        return Tensor(self.data+other.data)


    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

In [4]:
a = Tensor([1,2,3,4,5],autograd=True)
b = Tensor([2,2,2,2,2],autograd=True)
c = Tensor([5,4,3,2,1],autograd=True)

In [5]:
d = a+b
e = b+c
f = d+e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]
